# Banded Scaled Hashes

This is an initial exploration of "banded" scaled hash objects, which for a given `scaled` value and number of bands, divides the scaled hash space further into that number of bands.

This is a way of partitioning k-mer space which has some interesting consequences - warning, naive thoughts ahead :)

* band-by-band comparisons are individually smaller
* downsampling can simply ignore most of the bands
* overlap calculations can be truncated if the sum of the bands so far is larger than the threshold
* it's possible to parallelize comparisons of banded scaled hash objects by the number of bands
* there's a lot of interesting ways to make SBT/LCA databases smaller or more parallel

In [1]:
import sourmash
from sourmash import minhash

In [2]:
ls sigs/

GCF_000005845.2_ASM584v2_genomic.fna.gz.sig
GCF_000006945.1_ASM694v1_genomic.fna.gz.sig
GCF_000783305.1_ASM78330v1_genomic.fna.gz.sig


In [3]:
ss = sourmash.load_one_signature('sigs/GCF_000005845.2_ASM584v2_genomic.fna.gz.sig')
mh1 = ss.minhash

ss = sourmash.load_one_signature('sigs/GCF_000006945.1_ASM694v1_genomic.fna.gz.sig')
mh2 = ss.minhash

In [4]:
print(len(mh1), mh1.scaled)
print(len(mh2), mh2.scaled)

4476 1000
4900 1000


In [5]:
class BandedScaledMinHash:
    "Split a Scaled MinHash object into num_bands bands of hash space."
    def __init__(self, ksize, scaled, num_bands):
        self.max_hash = minhash._get_max_hash_for_scaled(scaled)
        self.band_size = minhash._get_max_hash_for_scaled(scaled * num_bands)
        self.empty_mh = minhash.MinHash(0, ksize, scaled=scaled)
        
        # subdivide the 'scaled' space into num_bands intervals.
        ivals = []
        last_start = 0
        for start in range(self.band_size, self.max_hash, self.band_size):
            ivals.append((last_start, start, self.empty_mh.copy()))
            last_start = start
        ivals.append((last_start, self.max_hash, self.empty_mh.copy()))

        self.ivals = ivals
        
    @classmethod
    def from_mh(cls, other_mh, num_bands):
        assert isinstance(other_mh, minhash.MinHash)
        obj = cls(other_mh.ksize, other_mh.scaled, num_bands)
        
        for hashval in other_mh.hashes:
            for (start, stop, band_mh) in obj.ivals:
                if hashval >= start and hashval < stop:
                    band_mh.add_hash(hashval)
                    break
                    
        return obj

    def intersection(self, other_mh):
        if isinstance(other_mh, BandedScaledMinHash):
            assert other_mh.max_hash == self.max_hash
            assert other_mh.band_size == self.band_size
            
            result = self.empty_mh.copy()
            
            for n in range(len(self.ivals)):
                result += self.ivals[n][2] & other_mh.ivals[n][2]
            return result
        elif isinstance(other_mh, minhash.MinHash):
            assert other_mh._max_hash == self.max_hash
            result = other_mh.copy_and_clear()

            for (_, _, band_mh) in self.ivals:
                result |= other_mh & band_mh
            return result
        assert 0
    __and__ = intersection

    def union(self, other_mh):
        if isinstance(other_mh, BandedScaledMinHash):
            assert other_mh.max_hash == self.max_hash
            assert other_mh.band_size == self.band_size
            
            result = self.empty_mh.copy()
            
            for n in range(len(self.ivals)):
                result += self.ivals[n][2] | other_mh.ivals[n][2]
            return result
        elif isinstance(other_mh, minhash.MinHash):
            assert other_mh._max_hash == self.max_hash
            result = other_mh.copy_and_clear()

            for (_, _, band_mh) in self.ivals:
                result |= other_mh | band_mh
            return result
        assert 0
    __or__ = union
    
    def __len__(self):
        return sum([ len(x) for (_, _, x) in self.ivals ])
    
    def similarity(self, other):
        return len(self & other) / len(self | other)
    
    def contained_by(self, other):
        return len(self & other) / len(self)
    
    def max_containment(self, other):
        numerator = len(self & other)
        denom1 = len(self)
        denom2 = len(other)
        return numerator / min(denom1, denom2)

In [6]:
bh1 = BandedScaledMinHash.from_mh(mh1, 10)
bh2 = BandedScaledMinHash.from_mh(mh2, 10)

In [7]:
assert bh1 & bh2  == mh1 & mh2
assert bh1 & mh2 == mh1 & mh2

assert bh1 | bh2  == mh1 | mh2
assert bh1 | mh2 == mh1 | mh2

In [8]:
assert bh1.similarity(bh2) == mh1.similarity(mh2)
assert bh1.similarity(mh2) == mh1.similarity(mh2)

In [9]:
assert bh1.contained_by(bh2) == mh1.contained_by(mh2)
assert bh1.contained_by(mh2) == mh1.contained_by(mh2)

assert bh2.contained_by(bh1) == mh2.contained_by(mh1)
assert bh2.contained_by(mh1) == mh2.contained_by(mh1)

In [10]:
assert bh1.max_containment(bh2) == mh1.max_containment(mh2)
assert bh1.max_containment(mh2) == mh1.max_containment(mh2)